In [1]:
import pandas as pd
import os
import sys

In [29]:
# Set paths
homey = os.getcwd()
redouxPath = os.path.join(homey, 'ForecastRedoux')
sqlPath = os.path.join(redouxPath, 'SQL')
rawDataPath = os.path.join(redouxPath, 'RawData')
AdditionalInfoPath = os.path.join(homey, 'AdditionalInfo')

In [3]:
# Pull in most recent timeline
timeline = pd.read_excel(os.path.join(homey, 'RegularForecast.xlsx'), sheetname='Timeline')

In [4]:
# Connect to FB API
sys.path.insert(0, 'Z:\Python projects\FishbowlAPITestProject')
import connecttest

In [5]:
# Query part costs
myresults = connecttest.create_connection(sqlPath, 'AvgCostQuery.txt')
myexcel = connecttest.makeexcelsheet(myresults)
connecttest.save_workbook(myexcel, rawDataPath, 'AvgCosts.xlsx')

Success!


In [6]:
# Get part costs into a dataFrame
costpath = os.path.join(rawDataPath, 'AvgCosts.xlsx')
avgCost = pd.read_excel(costpath) #Opens and puts the data into a dataframe
avgCost = avgCost.sort_values(by='PART', ascending=True) #Sort the data by part

In [7]:
# Merge part costs onto timeline
timeline = pd.merge(timeline.copy(), avgCost.copy(), how='left', on='PART')

In [9]:
# Calculate total costs
timeline['TotalCost'] = timeline['QTYREMAINING'] * timeline['AvgCost']

In [42]:
# Grab list of imaginary builds and create build number reference
newbuilds = pd.read_excel(os.path.join(AdditionalInfoPath, 'PartsToBuild.xlsx'), sheetname='Sheet1')
newbuilds.reset_index(inplace=True)
newbuilds['buildIndex'] = (newbuilds['index'] * -1) -1
newbuilds.drop('index', axis=1, inplace=True)

In [57]:
# Create columns to be filled by the next loop.
newbuilds['Total_Purchase'] = 0
newbuilds['Total_Cost_Produced'] = 0

In [119]:
newbuilds

,Part,Qty,Date,buildIndex,Total_Purchase,Total_Cost_Produced
0,016-1632-10 r01,38,2017-05-15,-1,16413,-30283
1,016-452-10 r03,54,2017-05-15,-2,2864,-17693
2,016-1622-10 r02,35,2017-05-15,-3,837,-12269
3,016-1922-10 r02,40,2017-05-15,-4,205872,-195876
4,016-1009-10 r02,40,2017-05-15,-5,23351,-57566
5,016-131-10,50,2017-05-15,-6,22113,-24885
6,016-1877-10 r03,370,2017-05-15,-7,183123,-196660
7,016-143-10,30,2017-05-15,-8,10377,-17536
8,016-138-10,20,2017-05-15,-9,3795,-9437
9,016-1847-10 r01,10,2017-05-15,-10,8629,-6391


In [118]:
""" This loop calculates and updates the total purchasing need for each imaginary build
    and it also finds the total raw material cost. """
for index, row in newbuilds.iterrows():
    imagBuild = row['buildIndex']
    thisBuild = timeline[timeline['GRANDPARENT'] == imagBuild].copy()
    purchDF = thisBuild[thisBuild['ORDERTYPE'] == 'Purchase'].copy()
    totPurchDF = purchDF[purchDF['TotalCost'].notnull()].copy()
    newbuilds.set_value(index, 'Total_Purchase', purchDF['TotalCost'].sum())
    # Now adding the total raw good cost of "Buy" items to show total amount in movement
    buyItemsDF = thisBuild[thisBuild['Make/Buy'] == 'Buy'].copy()
    rawGoodsDF = buyItemsDF[buyItemsDF['ORDERTYPE'] == 'Raw Good']
    newbuilds.set_value(index, 'Total_Cost_Produced', rawGoodsDF['TotalCost'].sum())
    # to be saved for lack of costing info, currently not used!!!
    noCostDF = rawGoodsDF[rawGoodsDF['TotalCost'].isnull()].copy() # ... it's so nice ...
    # what a waste, eh?

In [96]:
timeline

,ORDER,ITEM,ORDERTYPE,PART,QTYREMAINING,INV,DATESCHEDULED,PARENT,Make/Buy,GRANDPARENT,DESCRIPTION,AvgCost,TotalCost
0,0,0,Starting Inventory,007-007-10,0.0,37.0,1999-12-31 00:00:00.000,0,0,0,Panel Air Filter - 16 x 20; H2Zero (Nominal si...,16.249008,0.000000
1,748,Phantom,Raw Good,007-007-10,-1.0,36.0,2017-03-31 00:00:00.000,282,Buy,47293,Panel Air Filter - 16 x 20; H2Zero (Nominal si...,16.249008,-16.249008
2,-891,Imaginary,Raw Good,007-007-10,-10.0,26.0,2017-05-13 00:00:00.000,-369,Buy,-13,Panel Air Filter - 16 x 20; H2Zero (Nominal si...,16.249008,-162.490075
3,0,0,Starting Inventory,010-101-10,0.0,0.0,1999-12-31 00:00:00.000,0,0,0,2500VA Modular Inverter; TSI-EPC; -48Vdc; -120...,NaN,NaN
4,73200,1,Purchase,010-101-10,14.0,14.0,2017-03-24 00:00:00.000,73200,Buy,45623,2500VA Modular Inverter; TSI-EPC; -48Vdc; -120...,NaN,NaN
5,0,0,Starting Inventory,010-102-10,0.0,0.0,1999-12-31 00:00:00.000,0,0,0,Monitoring Unit; TSI-T2S-ETH; 24/48v-Media-48v...,NaN,NaN
6,73200,2,Purchase,010-102-10,3.0,3.0,2017-03-24 00:00:00.000,73200,Buy,0,Monitoring Unit; TSI-T2S-ETH; 24/48v-Media-48v...,NaN,NaN
7,0,0,Starting Inventory,010-103-10,0.0,0.0,1999-12-31 00:00:00.000,0,0,0,Control Panel; TSI-CATENA-STD-GUI; Door Mount ...,NaN,NaN
8,73200,3,Purchase,010-103-10,1.0,1.0,2017-03-24 00:00:00.000,73200,Buy,48168,Control Panel; TSI-CATENA-STD-GUI; Door Mount ...,NaN,NaN
9,0,0,Starting Inventory,010-104-10,0.0,0.0,1999-12-31 00:00:00.000,0,0,0,Shelf for Inverter Module; 19; Bravo-EPC-48V-1...,NaN,NaN


In [103]:
# using this as temporary variable to iterate through later
imagBuild = -1

In [104]:
thisBuild = timeline[timeline['GRANDPARENT'] == imagBuild].copy()

In [105]:
buyItemsDF = thisBuild[thisBuild['Make/Buy'] == 'Buy'].copy()

In [106]:
rawGoodsDF = buyItemsDF[buyItemsDF['ORDERTYPE'] == 'Raw Good']

In [107]:
noCostDF = rawGoodsDF[rawGoodsDF['TotalCost'].isnull()].copy()

In [114]:
rawGoodsDF['TotalCost'].sum()

-30283.83162838915

In [90]:
thisBuild = timeline[timeline['GRANDPARENT'] == imagBuild].copy()

In [91]:
purchDF = thisBuild[thisBuild['ORDERTYPE'] == 'Purchase'].copy()

In [92]:
noCostDF = purchDF[purchDF['TotalCost'].isnull()].copy()

In [93]:
totPurchDF = purchDF[purchDF['TotalCost'].notnull()].copy()

In [94]:
totPurchDF['TotalCost'].sum()

205872.9243432866

In [64]:
totPurchDF

,ORDER,ITEM,ORDERTYPE,PART,QTYREMAINING,INV,DATESCHEDULED,PARENT,Make/Buy,GRANDPARENT,DESCRIPTION,AvgCost,TotalCost
596,-219,Imaginary,Purchase,030-1069-10 r01,44.00000,76.00000,2017-05-14,-1,Buy,-1,GMT C14 Dual; Input Bus Screw Cover,0.481184,21.172095
625,-220,Imaginary,Purchase,030-1108-10 r01,36.00000,38.00000,2017-05-14,-1,Buy,-1,Glastic; Return Bus Isolator; GMT C14,5.500000,198.000000
2783,-2181,Imaginary,Purchase,460-045-10,64.00000,154.00000,2017-05-13,-233,Buy,-1,Fuse; PCB Resettable PTC; .20A; AC-72V; DC-72V...,0.184123,11.783890
3045,-1499,Imaginary,Purchase,520-034-10,1344.00000,1904.00000,2017-05-13,-233,Buy,-1,Fuse holder; GMT hldr; 125v; 20amp; pcb mnt w/...,1.208713,1624.510431
4212,-2534,Imaginary,Purchase,561-008-10,76.00000,76.00000,2017-05-14,-1,Buy,-1,Standoff insulators 1 standoff; 2165-1A,0.606493,46.093450
4961,-222,Imaginary,Purchase,590-1815-10 r03,27.00000,38.00000,2017-05-14,-1,Buy,-1,GMT C14 Chassis; Single; Dual Input,48.070000,1297.890000
4964,-223,Imaginary,Purchase,590-1819-10 r02,27.00000,38.00000,2017-05-14,-1,Buy,-1,GMT C14; Single; Dual Input Chassis Cover,17.790000,480.330000
4969,-224,Imaginary,Purchase,590-1829-10 r02,25.00000,38.00000,2017-05-14,-1,Buy,-1,GMT C14 Dual Input; A Side Return Bus Tier Three,12.730000,318.250000
4972,-225,Imaginary,Purchase,590-1830-10 r02,25.00000,38.00000,2017-05-14,-1,Buy,-1,GMT C14 Dual Input; B Side Return Bus Tier Three,9.730000,243.250000
4975,-226,Imaginary,Purchase,590-1831-10 r02,16.00000,38.00000,2017-05-14,-1,Buy,-1,GMT C14 Dual Input; B Side Return Bar,12.400000,198.400000


In [74]:
newbuilds.set_value(0, 'Total_Purchase', totPurchDF['TotalCost'].sum())

,Part,Qty,Date,buildIndex,Total_Purchase,Total_Cost_Produced
0,016-1632-10 r01,38,2017-05-15,-1,16413,0
1,016-452-10 r03,54,2017-05-15,-2,0,0
2,016-1622-10 r02,35,2017-05-15,-3,0,0
3,016-1922-10 r02,40,2017-05-15,-4,0,0
4,016-1009-10 r02,40,2017-05-15,-5,0,0
5,016-131-10,50,2017-05-15,-6,0,0
6,016-1877-10 r03,370,2017-05-15,-7,0,0
7,016-143-10,30,2017-05-15,-8,0,0
8,016-138-10,20,2017-05-15,-9,0,0
9,016-1847-10 r01,10,2017-05-15,-10,0,0
